In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as c
from astropy.wcs import WCS
import astropy.coordinates as coord
import astropy.units as u
from pyutils import *
import astropy.io.fits as fits
import healpy as hp
from astropy.table import Table

%load_ext autoreload
%autoreload 2

In [ ]:
ROOT_FOLDER = "../bin/"
FILE = ROOT_FOLDER + "BGS_BRIGHT_full.dat.fits"

In [ ]:
BGS_N = hp.read_map(ROOT_FOLDER + 'BGS_BRIGHT_mapprops_healpix_nested_nside256_N.fits')
BGS_S = hp.read_map(ROOT_FOLDER + 'BGS_BRIGHT_mapprops_healpix_nested_nside256_S.fits')

In [ ]:
hp.mollview(
    BGS_N,
    coord=["G", "E"],
    title="Histogram equalized Ecliptic",
    unit="Galaxies",
    norm="hist",
    min=0,
    max=1,
)
hp.graticule()

In [ ]:
hp.mollview(
    BGS_S,
    coord=["G", "E"],
    title="Histogram equalized Ecliptic",
    unit="Galaxies",
    norm="hist",
    min=0,
    max=1,
)
hp.graticule()

In [ ]:
with fits.open(FILE) as hdul:
    hdul.info()
    data = hdul[1].data
    header = (hdul[1].header)

   We need:

   'app_mag' CALCULATE IT. Find info for the bands on how to convert. 𝑚=22.5−2.5log10(flux) probably?

   'dec' GOT IT

   'g_r' GOT IT (but k-corr?)

   'ra' GOT IT

   'z_obs' GOT IT

   If warnings say multi fits, consider it LOST. We can actually use that info later if available.

Healpix for geometric info, hopefully containing 3pass info.

   fiber info

In [ ]:
columns_to_keep = ['TARGETID', 'BGS_TARGET', 'TARGET_STATE', 'LOCATION', 'SPECTYPE', 'Z_not4clus', 'ZERR', 'ZWARN', 'ZWARN_MTL', 'FLUX_R', 'FLUX_G', 'RA', 'DEC', 'BITWEIGHTS', 'PROB_OBS', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_G']

# There are also:
# FIBERFLUX_R/G/Z 
# FLUX_W1/2 
# MASKBITS ?
# ABSMAG_SDSS_G/R
# WEIGHT_ZFAIL

for c in columns_to_keep:
    print (c, data[3][c])

In [ ]:
u_table = Table.read(FILE, format='fits')


In [ ]:
plt.figure(figsize=(32,8))
plt.hist(u_table['TARGET_STATE'])
plt.title('Target State')
plt.yscale('log')

In [ ]:
plt.hist(u_table['SPECTYPE'])
plt.title("SPECTYPE")
plt.yscale('log')

## Get the galaxy data we actually need

In [ ]:
# TODO ensure this is right
def get_app_mag(FLUX_R):
    return 22.5 - 2.5*np.log10(FLUX_R)

In [ ]:
obj_type = u_table['SPECTYPE']
dec = u_table['DEC']
ra = u_table['RA']
z_obs = u_table['Z_not4clus']
target_id = u_table['TARGETID']
flux_r = u_table['FLUX_R']
flux_g = u_table['FLUX_G']
app_mag = get_app_mag(u_table['FLUX_R'])
p_obs = u_table['PROB_OBS']

before_count = len(dec)

#g_r = app_mag - get_app_mag(u_table['FLUX_G']) # TODO k-corr
#central = u_table['CEN']
#uchuu_halo_mass = u_table['HALO_MASS']

# TODO observed vs unobserved
# TODO 3pass only

redshift_filter = z_obs > 0
galaxy_filter = obj_type == 'GALAXY' # TODO what about the ones with no spectype???
app_mag_filter = app_mag < 19.5
#three_pass_filter = TODO
keep = np.all([galaxy_filter, app_mag_filter, redshift_filter], axis=0)
obj_type = obj_type[keep]
dec = dec[keep]
ra = ra[keep]
z_obs = z_obs[keep]
target_id = target_id[keep]
flux_r = flux_r[keep]
flux_g = flux_g[keep]
app_mag = app_mag[keep]
p_obs = p_obs[keep]

after_count = len(dec)

print(f"Kept {after_count} of {before_count} rows.")

In [ ]:
print(np.min(z_obs))
print(np.max(z_obs))

In [ ]:
plt.hist(u_table['ZWARN'])
plt.title('ZWARN')
plt.yscale('log')

In [ ]:
plt.hist(u_table['ZWARN_MTL'])
plt.title('ZWARN_MTL')
plt.yscale('log')

In [ ]:
plt.hist(z_obs, bins=50)
plt.title('Redshift')
plt.yscale('log')

In [ ]:
plt.hist(p_obs, bins=50)
plt.title('Probabilty to Observe')

In [ ]:
plt.hist(app_mag, bins=50)
plt.yscale('log')

In [ ]:
plt.hist(get_app_mag(u_table['FLUX_G']) - get_app_mag(u_table['FLUX_R']) , bins=20)
plt.title('g-r')
plt.yscale('log')

In [ ]:
print(estimate_frac_area(ra, dec))

In [ ]:
fig = make_map(ra, dec)